In [1]:
from sqlalchemy import inspect

from models import Base

In [2]:
# --- Schema Description Utility ---
def get_schema_description(base):
    """
    Generates a textual description of the database schema from SQLAlchemy ORM models.
    """
    description = "Database Schema:\n"
    mapper_registry = base.registry.mappers
    for mapper in mapper_registry:
        cls = mapper.class_
        table = cls.__table__
        description += f"\nTable: {table.name}\n"
        description += f"  Mapped Class: {cls.__name__}\n"
        description += "  Columns:\n"
        for column in table.columns:
            col_type = str(column.type)
            constraints = []
            if column.primary_key:
                constraints.append("PRIMARY KEY")
            for fk in column.foreign_keys:
                constraints.append(
                    f"REFERENCES {fk.column.table.name}({fk.column.name})"
                )
            if not column.nullable and not column.primary_key:
                constraints.append("NOT NULL")
            if column.unique:
                constraints.append("UNIQUE")
            description += f"    - {column.name} ({col_type})"
            if constraints:
                description += f" [{', '.join(constraints)}]"
            description += "\n"
        inspector = inspect(cls)
        if inspector.relationships:
            description += "  Relationships:\n"
            for name, rel in inspector.relationships.items():
                target = rel.mapper.class_
                description += (
                    f"    - {name}: {rel.direction.name} -> {target.__tablename__} "
                    f"(Class: {target.__name__})\n"
                )
    return description

In [3]:
print(get_schema_description(Base))

Database Schema:

Table: users
  Mapped Class: User
  Columns:
    - id (INTEGER) [PRIMARY KEY]
    - username (VARCHAR) [NOT NULL, UNIQUE]
    - email (VARCHAR) [NOT NULL, UNIQUE]
    - created_at (DATETIME)

Table: user_subscriptions
  Mapped Class: UserSubscription
  Columns:
    - id (INTEGER) [PRIMARY KEY]
    - user_id (INTEGER) [REFERENCES users(id), NOT NULL]
    - subscription_id (INTEGER) [REFERENCES subscriptions(id), NOT NULL]
    - start_date (DATETIME)
    - end_date (DATETIME)
    - status (VARCHAR(9)) [NOT NULL]

Table: invoices
  Mapped Class: Invoice
  Columns:
    - id (INTEGER) [PRIMARY KEY]
    - user_subscription_id (INTEGER) [REFERENCES user_subscriptions(id), NOT NULL]
    - invoice_date (DATETIME) [NOT NULL]
    - amount (FLOAT) [NOT NULL]
    - status (VARCHAR) [NOT NULL]
    - created_at (DATETIME)

Table: subscriptions
  Mapped Class: Subscription
  Columns:
    - id (INTEGER) [PRIMARY KEY]
    - name (VARCHAR) [NOT NULL, UNIQUE]
    - price (FLOAT) [NOT NUL